In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 99% 85.0M/85.7M [00:01<00:00, 64.1MB/s]
100% 85.7M/85.7M [00:01<00:00, 73.9MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 13.4 MB/s 
     |████████████████████████████████| 1.2 MB 86.1 MB/s 
     |████████████████████████████████| 6.4 MB 63.6 MB/s 
     |████████████████████████████████| 120 kB 72.8 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 3.4 MB 69.6 MB/s 
     |████████████████████████████████| 1.1 MB 52.8 MB/s 
     |████████████████████████████████| 87 kB 8.9 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 234 kB 70.8 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 840 kB 61.3 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 13.2 MB/s 
     |████████████████████████████████| 47.7 MB 1.1 MB/s 
     |████████████████████████████████| 352 kB 80.5 MB/s 
     |████████████████████████████████| 462 kB 78.0 MB/s 
     |█████████████

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [ ]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [ ]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

In [11]:
spec = model_spec.get('efficientdet_lite4')

In [12]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=50, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/50
397/397 [==============================] - 617s 1s/step - det_loss: 0.9737 - cls_loss: 0.7961 - box_loss: 0.0036 - reg_l2_loss: 0.1090 - loss: 1.0827 - learning_rate: 0.0078 - gradient_norm: 3.7198 - val_det_loss: 0.6120 - val_cls_loss: 0.5459 - val_box_loss: 0.0013 - val_reg_l2_loss: 0.1094 - val_loss: 0.7214
Epoch 2/50
397/397 [==============================] - 553s 1s/step - det_loss: 0.5988 - cls_loss: 0.5098 - box_loss: 0.0018 - reg_l2_loss: 0.1097 - loss: 0.7085 - learning_rate: 0.0075 - gradient_norm: 3.5568 - val_det_loss: 0.5109 - val_cls_loss: 0.4271 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.1099 - val_loss: 0.6208
Epoch 3/50
397/397 [==============================] - 554s 1s/step - det_loss: 0.5354 - cls_loss: 0.4545 - box_loss: 0.0016 - reg_l2_loss: 0.1101 - loss: 0.6456 - learning_rate: 0.0075 - gradient_norm: 3.6252 - val_det_loss: 0.5043 - val_cls_loss: 0.4580 - val_box_loss: 9.2586e-04 - val_reg_l2_loss: 0.1104 - val_loss: 0.6146
Epoch 4/50
397/397 [=======

In [13]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 28202.00845527649


In [14]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 24s 246ms/step

{'AP': 0.8407219, 'AP50': 0.93246996, 'AP75': 0.8939441, 'APs': -1.0, 'APm': 0.58920294, 'APl': 0.85505164, 'ARmax1': 0.7674507, 'ARmax10': 0.88487846, 'ARmax100': 0.8895023, 'ARs': -1.0, 'ARm': 0.64166665, 'ARl': 0.90313053, 'AP_/10': 0.78154725, 'AP_/20': 0.85811657, 'AP_/50': 0.8308702, 'AP_/100': 0.93808603, 'AP_/200': 0.88287127, 'AP_/500': 0.8408019, 'AP_/2000': 0.75276023}


In [15]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 27.505277633666992


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [16]:
model.export(export_dir='/content/')